# **Implementing Many-to-Many RNN for English-to-Urdu Language Translation and Exploring Its Limitations**

# **Part 1:** Many-to-Many Recurrent Neural Network (RNN) Implementation

## Data Preparation:

In [1]:
import pandas as pd


df = pd.read_excel('./parallel-corpus.xlsx')

# Keep only the first two columns
df = df.iloc[:, :2]

df.rename(columns = {'SENTENCES ':'SENTENCES'}, inplace = True)


df.head()

,SENTENCES,MEANING
0,How can I communicate with my parents?,میں اپنے والدین سے کیسے بات کروں ؟
1,How can I make friends?’,میں دوست کیسے بنائوں ؟
2,Why do I get so sad?’,میں اتنا اداس کیوں ہوں؟.
3,"If you’ve asked yourself such questions, you’r...",اگر آپ نے اپنے آپ سے ایسے سوالات کیے ہیں، تو آ...
4,"Depending on where you’ve turned for guidance,...",اس بات پر منحصر ہے کہ آپ رہنمائی کے لیے کہاں ...


Step 1: Preprocess the Data

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Tokenize the sentences
tokenizer_eng = Tokenizer()
tokenizer_urdu = Tokenizer()

# Convert the 'SENTENCES' column to string type before fitting the tokenizer
df['SENTENCES'] = df['SENTENCES'].astype(str)
# Convert the 'MEANING' column to string type before fitting the tokenizer
df['MEANING'] = df['MEANING'].astype(str)

tokenizer_eng.fit_on_texts(df['SENTENCES'])
tokenizer_urdu.fit_on_texts(df['MEANING'])

eng_sequences = tokenizer_eng.texts_to_sequences(df['SENTENCES'])
urdu_sequences = tokenizer_urdu.texts_to_sequences(df['MEANING'])

# Pad sequences
max_len_eng = max(len(seq) for seq in eng_sequences)
max_len_urdu = max(len(seq) for seq in urdu_sequences)

max_len = max(max_len_eng,max_len_urdu)

eng_sequences = pad_sequences(eng_sequences, maxlen=max_len, padding='post')
urdu_sequences = pad_sequences(urdu_sequences, maxlen=max_len, padding='post')

# Vocabulary sizes
vocab_size_eng = len(tokenizer_eng.word_index) + 1
vocab_size_urdu = len(tokenizer_urdu.word_index) + 1

Step 2: Prepare the Data for Training

In [3]:
# Split the data into training and validation sets
train_size = int(len(eng_sequences) * 0.7)
test_size = int(len(eng_sequences) * 0.15)

# For English train, validation and test
x_train, x_temp = eng_sequences[:train_size], eng_sequences[train_size:]
x_test, x_val = x_temp[:test_size], x_temp[test_size:]

# For Urdu train, validation and test
y_train, y_temp = urdu_sequences[:train_size], urdu_sequences[train_size:]
y_test, y_val = y_temp[:test_size], y_temp[test_size:]


In [4]:
print(x_train.shape, y_train.shape )

(21114, 938) (21114, 938)


Step 3: Build the RNN Model

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense


# Build Model
model = Sequential(
    [
        Embedding(input_dim=vocab_size_eng,output_dim=64),
        SimpleRNN(64, return_sequences=True),
        Dense(vocab_size_urdu, activation='softmax'),
    ]
)


In [6]:
# Compile Model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train Model
model.fit(x_train, y_train, epochs=25,validation_data=(x_val,y_val))


Epoch 1/25
660/660 ━━━━━━━━━━━━━━━━━━━━ 204s 297ms/step - accuracy: 0.9633 - loss: 2.2587 - val_accuracy: 0.9873 - val_loss: 0.1036
Epoch 2/25
660/660 ━━━━━━━━━━━━━━━━━━━━ 250s 288ms/step - accuracy: 0.9833 - loss: 0.1379 - val_accuracy: 0.9880 - val_loss: 0.0964
Epoch 3/25
660/660 ━━━━━━━━━━━━━━━━━━━━ 190s 288ms/step - accuracy: 0.9834 - loss: 0.1334 - val_accuracy: 0.9881 - val_loss: 0.0919
Epoch 4/25
660/660 ━━━━━━━━━━━━━━━━━━━━ 197s 281ms/step - accuracy: 0.9836 - loss: 0.1294 - val_accuracy: 0.9883 - val_loss: 0.0888
Epoch 5/25
660/660 ━━━━━━━━━━━━━━━━━━━━ 202s 281ms/step - accuracy: 0.9838 - loss: 0.1271 - val_accuracy: 0.9885 - val_loss: 0.0859
Epoch 6/25
660/660 ━━━━━━━━━━━━━━━━━━━━ 190s 288ms/step - accuracy: 0.9841 - loss: 0.1231 - val_accuracy: 0.9885 - val_loss: 0.0851
Epoch 7/25
660/660 ━━━━━━━━━━━━━━━━━━━━ 197s 281ms/step - accuracy: 0.9843 - loss: 0.1188 - val_accuracy: 0.9886 - val_loss: 0.0831
Epoch 8/25
660/660 ━━━━━━━━━━━━━━━━━━━━ 207s 288ms/step - accuracy: 0.9842 -

In [8]:
model.evaluate(x_test,y_test)

142/142 ━━━━━━━━━━━━━━━━━━━━ 19s 130ms/step - accuracy: 0.9852 - loss: 0.1073


[0.07844744622707367, 0.9887716770172119]

In [21]:
# Translate function
def translate(text):
    sequence = tokenizer_eng.texts_to_sequences([text])
    sequence = pad_sequences(sequence, maxlen=max_len, padding='post')
    prediction = model.predict(sequence)
    predicted_sequence = np.argmax(prediction, axis=-1)
    translated_text = ' '.join([tokenizer_urdu.index_word[idx] for idx in predicted_sequence[0] if idx != 0])
    return translated_text


In [18]:
print(translate('How can I make friends?'))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
آپ آپ میں میں


In [22]:
for sentence in df.head()['SENTENCES']:
  print(sentence)
  print(translate(sentence))

How can I communicate with my parents?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
آپ آپ میں میں میں میں کے
How can I make friends?’
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
آپ آپ میں میں
Why do I get so sad?’
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
آپ آپ میں میں
If you’ve asked yourself such questions, you’re not alone.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
اگر آپ نے کو آپ نے کے کو کہ کے
Depending on where you’ve turned for guidance, you may have been given conflicting answers.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
اس کے آپ آپ آپ کے آپ آپ آپ کے نہیں سے کے سے کے


In [23]:
model.save("english_urdu_RNN_f219258.keras")